In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Celiac_Disease/GSE113469'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Identification of peculiar gene expression profile in peripheral blood mononuclear cells (PBMC) of celiac patients on gluten free diet"
!Series_summary	"The aim of the present study was to investigate changes in the whole-genome profile of the PBMCs of subjects with celiac disease (vs. controls)."
!Series_overall_design	"RNA obtained from PBMCs of the CTRL subjects and celiac patients. Comparisons: control vs celiac"
Sample Characteristics Dictionary:
{0: ['disease state: Healthy Control', 'disease state: Celiac Disease'], 1: ['age: 30', 'age: 27', 'age: 31', 'age: 26', 'age: 28', 'age: 32', 'age: 41', 'age: 34', 'age: 25', 'age: 42', 'age: 29', 'age: 21', 'age: 44', 'age: 56', 'age: 50', 'age: 51', 'age: 37'], 2: ['cell type: PBMCs']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check for gene expression data availability
is_gene_available = True  # based on the phrase "whole-genome profile of the PBMCs"

# Assign keys for 'Celiac_Disease', 'age', 'gender'
trait_row = 0  # 'disease state' information available
age_row = 1  # 'age' information available
gender_row = None  # 'gender' information is not available

# Conversion functions
def convert_trait(value):
    if 'Healthy Control' in value:
        return 0
    elif 'Celiac Disease' in value:
        return 1
    return None

def convert_age(value):
    try:
        age = int(value.split(':')[1].strip())
        return age
    except ValueError:
        return None

# Save cohort information
save_cohort_info('GSE113469', './preprocessed/Celiac_Disease/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Celiac_Disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Celiac_Disease/trait_data/GSE113469.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM3106457': [0, 30], 'GSM3106458': [0, 30], 'GSM3106459': [0, 27], 'GSM3106460': [0, 31], 'GSM3106461': [0, 26], 'GSM3106462': [0, 27], 'GSM3106463': [0, 28], 'GSM3106464': [0, 28], 'GSM3106465': [0, 31], 'GSM3106466': [0, 32], 'GSM3106467': [0, 41], 'GSM3106468': [0, 34], 'GSM3106469': [0, 30], 'GSM3106470': [0, 31], 'GSM3106471': [0, 25], 'GSM3106472': [0, 26], 'GSM3106473': [0, 42], 'GSM3106474': [0, 31], 'GSM3106475': [0, 31], 'GSM3106476': [0, 29], 'GSM3106477': [1, 27], 'GSM3106478': [1, 21], 'GSM3106479': [1, 21], 'GSM3106480': [1, 27], 'GSM3106481': [1, 44], 'GSM3106482': [1, 56], 'GSM3106483': [1, 41], 'GSM3106484': [1, 28], 'GSM3106485': [1, 28], 'GSM3106486': [1, 50], 'GSM3106487': [1, 51], 'GSM3106488': [1, 41], 'GSM3106489': [1, 32], 'GSM3106490': [1, 34], 'GSM3106491': [1, 37], 'GSM3106492': [1, 50], 'GSM3106493': [1, 25]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the keys storing the identifiers and gene symbols
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/Celiac_Disease/gene_data/GSE113469.csv'
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Celiac_Disease')

# If the trait is not severely biased, save the cohort information and the merged data.

# 4. Save the cohort information.
save_cohort_info('GSE113469', './preprocessed/Celiac_Disease/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/Celiac_Disease/GSE113469.csv'
    unbiased_merged_data.to_csv(csv_path)


For the feature 'Celiac_Disease', the least common label is '1.0' with 17 occurrences. This represents 45.95% of the dataset.
The distribution of the feature 'Celiac_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 27.0
  50% (Median): 31.0
  75%: 37.0
Min: 21.0
Max: 56.0
The distribution of the feature 'Age' in this dataset is fine.

